#### Reading from Firestore

Collections:

`topic_collection_ref = store.collection('junto').document(
        'living-room-convos-dev').collection('topics')`

Document:

`doc_ref = store.collection('junto').document('living-room-convos-dev')`

Collection Group (aggregates across all subgroups):

`store.collection_group('discussions')`

Limit Results:

`doc_ref = collection.limit(limit).get()`
        
Manipulate data:

`store.collection_group('discussions').stream() # Or get()
discussions = {d.id : d.to_dict() for d in docs}
`

# Setup

## Imports

In [ ]:
from datetime import date
from datetime import timedelta
from datetime import timezone
from datetime import datetime
import datetime as dt
import csv
from collections import defaultdict

import json
import requests
import random
import pandas as pd
from firebase_admin import credentials, firestore
import os.path
import pprint
from string import Template
import string
import numpy as np
import math
import enum
import re
import html2text
from google.cloud import storage
from gsheets import Sheets
import google.cloud
from firebase_admin import auth
import firebase_admin
import sys
import os

from twilio.rest import Client

print(os.path.dirname(sys.executable))


## Init Firebase

In [ ]:
JUNTO_DIR = '..'
PROD_CRED = credentials.Certificate(
    JUNTO_DIR + "/creds/ServiceAccountKey_prod.json")
DEV_CRED = credentials.Certificate(
    JUNTO_DIR + "/creds/ServiceAccountKey_dev.json")
BUCKET = 'juntochat.appspot.com'


def initialize_firestore(use_prod=False):
    if use_prod:
        app = firebase_admin.initialize_app(PROD_CRED)
    else:
        app = firebase_admin.initialize_app(DEV_CRED)
    return firestore.client()


store = initialize_firestore(use_prod=True)

# Helper Fns

In [ ]:
def get_docs(collection, limit=None):
    doc_ref = collection.limit(limit)
    try:
        return doc_ref.get()
    except google.cloud.exceptions.NotFound:
        print(u'Missing data')


def get_and_print_topics(collection_id):
    collection = store.collection('junto').document(
        collection_id).collection('topics')
    topics = get_docs(collection)
    for doc in topics:
        pprint.pprint(doc.to_dict(), compact=True)


def event_participants(junto_id, topic_id, discussion_id):
    event_doc = store.document(
        f'junto/{junto_id}/topics/{topic_id}/discussions/{discussion_id}')
    participant_collection = event_doc.collection('discussion-participants')
    return [p.id for p in participant_collection.stream()]


def get_publicusers(user_ids):
    '''Return dict of user_id --> PublicUser.'''
    publicusers = [u for u in store.collection(
        'publicUser').get() if u.id in user_ids]
    batch_size = 100
    result = {}
    uid_identifiers = [auth.UidIdentifier(m) for m in user_ids]
    for i in range(0, len(uid_identifiers), batch_size):
        # print("%d--> %d" % (i,i+batch_size))
        batch_result = {u.uid: u for u in auth.get_users(
            uid_identifiers[i:i+batch_size]).users}
        result.update(batch_result)
    return result


def get_suggestions(junto, topic, discussion, write_to=None):
    suggestions = store.collection('junto').document(junto).collection(
        'topics').document(topic).collection('discussions').document(discussion).collection('user-suggestions').get()
    if write_to:
        f = open(write_to, "w")
        f.write('\n'.join([s.to_dict()['content'] for s in suggestions]))
        f.close()
    return suggestions


def get_twilio_client():
    # Twilio Creds from twilio.com/console
    account_sid = "AC8e7bb5a5223120747ab835398c6e6634"
    TWILIO_CRED_FILE = '%s/creds/twilio_auth_token' % JUNTO_DIR
    auth_token  = open(TWILIO_CRED_FILE, "r").read().strip()
    client = Client(account_sid, auth_token)
    return client


'''
# Print all LRC topics
collection_id = 'living-room-convos-dev'
get_and_print_topics(collection_id)      

# Get all discussions, across all Juntos
docs = store.collection_group('discussions').stream() # OR use get()

# Map of discussion_id to discussion dict
discussions = {d.id : d.to_dict() for d in docs}

# Get all junto_ids
junto_ids = set([d.to_dict()['juntoId'] for d in docs])
'''


# Meetings per Junto

In [ ]:
'''
3 Locations:
* junto/<juntoId>/topics/<topicId>/discussions/<discussionId>/live-meetings/<discussionId>
* junto/<juntoId>/topics/<topicId>/discussions/<discussionId>/breakout-room-live-meeting/<breakoutRoomId>
* junto/<juntoId>/instant-live-meeting/<meetingId>
'''


In [ ]:
COLLECTION_GROUP_PATHS = ['live-meetings',
                          'breakout-room-live-meeting', 'instant-live-meeting']


def num_participants(d):
    print_missing_participants = False
    participant_count = 0
    participantId_count = 0
    if 'participants' in d and d['participants']:
        participant_count = len(d['participants'])
    if 'participantIds' in d and d['participantIds']:
        participantId_count = len(d['participantIds'])
    # if print_missing_participants:
    #     print(f'\nMISSING PARTICIPANTS - junto: {d["juntoId"]}')
    #     print(d)
    return max(participantId_count, participant_count)


meetings = []
for cg in COLLECTION_GROUP_PATHS:
    for m in store.collection_group(cg).stream():
        md = m.to_dict()
        md['meetingId'] = m.id
        md['juntoId'] = m.reference.path.split('/')[1]
        md['pcount'] = num_participants(md)
        meetings.append(md)


In [ ]:
# Using discussions
df = pd.DataFrame(meetings)
df[df['pcount'] > 1].groupby('juntoId').size().sort_values(ascending=False)

# Average Ratings

In [ ]:
# /junto/60GgYwIKJFyqmNUzUcfp/topics/GeG3jzfxOr61CQDLbngj/discussions/ot8QG9xaSuNh5lwqPoSL/live-meetings/ot8QG9xaSuNh5lwqPoSL/ratings
# <discussionPath>/<discussionId>/live-meetings/<discussionId>/ratings/<userId>
def ratingdict(r):
    d = r.to_dict()
    d['path'] = r.reference.path
    d['juntoId'] = d['path'].split('/')[1]
    return d


ratings = pd.DataFrame([ratingdict(r)
                        for r in store.collection_group('ratings').stream()])
print("%d ratings" % len(ratings))


In [ ]:
ratings.groupby('juntoId').count()


In [ ]:
ratings['rating'].mean()

# Breakout Report

In [ ]:
# Encryption stuff for America Talks
# See https://pycryptodome.readthedocs.io/en/latest/src/cipher/classic.html#cbc-mode
import itertools
from base64 import urlsafe_b64decode, urlsafe_b64encode
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad, pad


def encrypt(key, iv, data):
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ct_bytes = cipher.encrypt(pad(data, AES.block_size))
    ct = urlsafe_b64encode(ct_bytes).decode('utf-8')
    return ct

def decrypt(key, iv, answermask):
    try:
        ct = urlsafe_b64decode(answermask)
        cipher = AES.new(key, AES.MODE_CBC, iv)
        pt = unpad(cipher.decrypt(ct), AES.block_size)
        return pt.decode("utf-8")
    except (ValueError, KeyError):
        return "Incorrect decryption"

# # Encryption from AT
# key = b'\x1B\xB1\x0En3\xF9\xD0\xC5\xB8t\xB6\x8388\xFD+'
# iv = b'u\n\xD1mP\x1E\xF7\xF7Ah\xC6\x11\x1Bu#\b'

In [ ]:
def hamming(s1, s2):
    if len(s1) != len(s2):
        print(s1, s2)
        raise Exception('Strings are different lengths.')
    return sum(c1 != c2 for c1, c2 in zip(s1, s2))


def printDistribution(breakouts, samples):
    maxDist = len(samples[breakouts[0][0]])
    dists = [0]*(maxDist+1)
    nonPairs = 0
    pairsMissingParams = 0
    s = '\nDistribution: \n\t'
    for breakout in breakouts:
        if (len(breakout) > 3):
            nonPairs += 1
            print(f'WARNING: Room with {len(breakout)} participants.')
        elif len(breakout) != 2:
            nonPairs += 1
        elif breakout[0] in samples and breakout[1] in samples:
            dists[hamming(samples[breakout[0]], samples[breakout[1]])] += 1
        else:
            pairsMissingParams += 1
    s += '\n\t'.join([f'Dist {i}:  {v}' for (i, v) in enumerate(dists)])
    s += f'\n\tNonpairs:  {nonPairs}'
    s += f'\n\tPairs missing params:  {pairsMissingParams}'
    print(s)


def printPairs(breakouts, samples, participant_ids=None):
    s = []
    dists = []
    for p in breakouts:
        pstring = '\t'+' '.join([samples[pi] for pi in p])
        if participant_ids:
            pstring = '\t'+' '.join([participant_ids[pi] for pi in p])
        if len(p) == 2:
            d = hamming(samples[p[0]], samples[p[1]])
            s.append(f"{pstring}\t\tDistance: {d}")
            dists.append(d)
        else:
            s.append(pstring + ' '.join([samples[pi] for pi in p]))
    return '\n'.join(s)


In [ ]:
junto_id = 'allsides-talks'
topic_id = 'MBRiMvQyFcVe6d8hsPFI'
discussion_id = 'i-allsides-talks-smart-match-test-1'
event_doc = store.document(
    f'junto/{junto_id}/topics/{topic_id}/discussions/{discussion_id}')

# Get most recent breakout
breakout_sessions = event_doc.collection(
    'live-meetings').document(discussion_id).collection('breakout-room-sessions')
# Or '555c5170-c593-11eb-8444-d1a7c5b54f88'
breakout_session_id = breakout_sessions.get()[-1].id
print(f'Breakout Session ID: {breakout_session_id}')
breakout_room_docs = breakout_sessions.document(
    breakout_session_id).collection('breakout-rooms').get()
breakout_rooms = sorted(
    [r.to_dict() for r in breakout_room_docs], key=lambda r: r['orderingPriority'])

# Other stuff
discussion_participants = event_doc.collection('discussion-participants')


In [ ]:

def matchCodeRoom(participants):
    if len(participants) != 2:
        return False
    # print('Participants', participants)
    matchcodes = [parameters[p]['match_id'] for p in participants]
    # print(matchcodes)
    # if matchcodes[0] == matchcodes[1] and len(matchcodes[0])>1:
    #     return True
    return False


# Read parameters for each participant
samples = {}
parameters = {}
for participant in discussion_participants.stream():
    joinparameters = participant.to_dict()['joinParameters']
    if joinparameters and 'am' in joinparameters:
        # samples[participant.id] = decrypt(key, iv, joinparameters['am'])
        samples[participant.id] = joinparameters['am']
        parameters[participant.id] = joinparameters
    else:  # TODO:Add Back
        samples[participant.id] = None
        parameters[participant.id] = joinparameters


# Read breakout rooms
breakouts = []
participantIdField = 'originalParticipantIdsAssignment'
for room in breakout_rooms:
    if participantIdField not in room or not room[participantIdField]:
        continue
    if room['roomId'] == 'waiting-room':
        continue
    participants = [p for p in room[participantIdField]]
    if matchCodeRoom(participants):
        continue
    breakouts.append(participants)


printDistribution(breakouts, samples)
# participant_ids = {k:(['participant_id'] if 'participant_id' in v else '') for k,v in parameters.items()}
# pairs = printPairs(breakouts, samples, participant_ids = participant_ids)
# print('\nPairs:\n' + pairs)


In [ ]:
yes_params = 0
no_params = 0
for p in discussion_participants.stream():
    if p.id in samples:
        yes_params += 1
    else:
        no_params += 1

print(f"{yes_params} yes, {no_params} no")


In [ ]:
write_files = False
if write_files:
    f = open("/Users/benjaminturtel/Desktop/pairs_distances.txt", "w")
    f.write(pairs)
    f.close()
    f = open("/Users/benjaminturtel/Desktop/all_participants.txt", "w")
    f.write('\n'.join(participant_ids.values()))
    f.close()


# Event Participant Report

In [ ]:
def get_breakouts(junto, topic, discussion):
    event_doc = store.document(
        f'junto/{junto}/topics/{topic}/discussions/{discussion}')
    breakout_sessions = event_doc.collection(
        'live-meetings').document(discussion).collection('breakout-room-sessions')
    breakout_session_id = breakout_sessions.get()[-1].id  # Most recent
    print(f'Breakout Session ID: {breakout_session_id}')
    breakout_room_docs = breakout_sessions.document(
        breakout_session_id).collection('breakout-rooms').get()
    breakout_rooms = sorted(
        [r.to_dict() for r in breakout_room_docs], key=lambda r: r['orderingPriority'])
    return breakout_rooms


def get_breakout_dicts(junto, topic, discussion):
    ORIGINAL_PIDS = 'originalParticipantIdsAssignment'
    ROOM_NAME = 'roomName'
    d = {}
    assigned_breakout = {}
    final_breakout = {}
    breakouts = get_breakouts(junto, topic, discussion)
    for b in breakouts:
        if b[ORIGINAL_PIDS]:
            for p in b[ORIGINAL_PIDS]:
                if p in assigned_breakout:
                    print(p, assigned_breakout[p], b[ROOM_NAME])
                assigned_breakout[p] = b[ROOM_NAME]
        for p in b['participantIds']:
            if p in final_breakout:
                print(p, final_breakout[p], b[ROOM_NAME])
            final_breakout[p] = b[ROOM_NAME]
    return assigned_breakout, final_breakout


def get_join_params(junto, topic, discussion, paramkeys):
    event_doc = store.document(
        f'junto/{junto}/topics/{topic}/discussions/{discussion}')
    participant_collection = event_doc.collection('discussion-participants')
    params = defaultdict(list)
    for p in participant_collection.stream():
        params['junto_pid'].append(p.id)
        pdict = p.to_dict()
        joinparams = {}
        if 'joinParameters' in pdict and pdict['joinParameters']:
            joinparams = pdict['joinParameters']
        for k in paramkeys:
            if k in joinparams:
                params[k].append(joinparams[k])
            else:
                params[k].append(None)
    return params


def get_subscribed_set(junto, participants):
    participantset = set(participants)
    subscribed = set()
    for m in store.collection_group('junto-membership').stream():
        member = m.to_dict()
        if member['juntoId'] == junto and member['userId'] in participantset:
            subscribed.add(member['userId'])
    return subscribed


def event_dataframe(junto, topic, discussion):
    event_doc = store.document(
        f'junto/{junto}/topics/{topic}/discussions/{discussion}')
    params = get_join_params(junto, topic, discussion, [
                             'match_id', 'am', 'participant_id'])
    assigned_breakout, final_breakout = get_breakout_dicts(
        junto, topic, discussion)
    publicusers = get_publicusers(params['junto_pid'])
    subscribed = get_subscribed_set(junto, params['junto_pid'])
    df = pd.DataFrame(params)
    df['name'] = df['junto_pid'].map(lambda x: publicusers[x].display_name)
    df['email'] = df['junto_pid'].map(lambda x: publicusers[x].email)
    df['subscribed'] = df['junto_pid'].isin(subscribed)
    df['assigned_breakout'] = df['junto_pid'].map(assigned_breakout)
    df['final_breakout'] = df['junto_pid'].map(final_breakout)
    df['domain'] = df['email'].map(lambda x: x.split('@')[1] if x else '')
    return df


In [ ]:
junto = 'allsides-talks'
topic = 'kGZFGLnLL9AsGSdPAnWX'
discussion = '2d1vxGI7ZAt3lhydfCqN'

df = event_dataframe(junto, topic, discussion)

In [ ]:
# Write report
dirpath = f"/Users/benjaminturtel/Desktop/reports/"
if not os.path.exists(dirpath):
    os.makedirs(dirpath)

tag = f'{junto}_{topic}_{discussion}'
write_path = f"{dirpath}/{tag}.csv"
df.to_csv(write_path, index=False, columns=['name', 'email', 'participant_id', 'assigned_breakout', 'final_breakout', 'am'])

suggestions = get_suggestions(junto, topic, discussion, f"{dirpath}/{tag}_suggestions.csv")


In [ ]:
df

# Event Rooms Report

In [ ]:
def get_twilio_room(unique_name):
    # TODO: Remove date filter
    # print(unique_name)
    twilio = get_twilio_client()
    trs = twilio.video.rooms.list(limit=10, unique_name=unique_name, status='completed')
    # Participants > 1 first, then by duration
    return sorted(trs, key=lambda r: (len(r.participants.list()) > 1, r.duration), reverse=True)[0]

def get_breakout_dataframe(junto, topic, discussion):
    breakouts = get_breakouts(junto, topic, discussion)[1:]
    rooms = pd.DataFrame({'name': [b['roomName'] for b in breakouts],
                            'roomId': [b['roomId'] for b in breakouts]})
    twilio_rooms = [get_twilio_room(sid)
                    for sid in rooms['roomId']]  # This takes time
    rooms['duration'] = [r.duration/60 for r in twilio_rooms]
    rooms['sid'] = [r.sid for r in twilio_rooms]
    rooms['num_participants'] = [len(set([p.identity for p in r.participants.list()]))
                                     for r in twilio_rooms]
    return rooms


In [ ]:
junto = 'meetingofamerica'
topic = 'mjfIEQbtNrqtqWHyvzgL'
event = 'cZ4bzX2uXtLnxJQRyYMR'

In [ ]:
breakouts = get_breakouts(junto, topic, event)[1:]
rooms = pd.DataFrame({'name': [b['roomName'] for b in breakouts],
                        'roomId': [b['roomId'] for b in breakouts]})
twilio_rooms = [get_twilio_room(sid)
                for sid in rooms['roomId']]  # This takes time
rooms['duration'] = [r.duration/60 for r in twilio_rooms]
rooms['sid'] = [r.sid for r in twilio_rooms]
rooms['num_participants'] = [len(r.participants.list())
                                for r in twilio_rooms]

rooms = get_breakout_dataframe(junto, topic, discussion)


# MOA Stats

In [ ]:
# Get collection of discussions
# https://myjunto.app/home/junto/meetingofamerica/discuss/mjfIEQbtNrqtqWHyvzgL/YKjTPlQdR8fK4wVRjovn
discussion_collection = store.collection('junto').document('meetingofamerica').collection('topics').document(
    'mjfIEQbtNrqtqWHyvzgL').collection('discussions')
discussions = get_docs(discussion_collection)  # Get discussions
participants = []
for disc in discussions:  # Get participant count for each discussion
    for doc in discussion_collection.document(disc.id).collection('discussion-participants').stream():
        dd = doc.to_dict()
        p = {}
        p['pid'] = doc.id
        jp =  dd['joinParameters']
        if jp and 'moa-partner' in jp:
            p['partner'] = jp['moa-partner']
        else:
            p['partner'] = None
        sq = dd['breakoutRoomSurveyQuestions']
        for i, q in enumerate(sq):
            p['a%d'%i] = q['answerIndex']
        participants.append(p)
df = pd.DataFrame(participants)
df

# AllSides Stats
Calculate how many conversations with how many participants

In [ ]:
def get_allsides_stats():
    # Get collection of discussions
    discussion_collection = store.collection('junto').document('allsides-talks').collection('topics').document(
        'NVwyIgtNhz6frXWrlTrC').collection('discussions')
    discussions = get_docs(discussion_collection)  # Get discussions
    records = []
    for disc in discussions:  # Get participant count for each discussion
        records.append([
            disc.id,
            disc.to_dict()['scheduledTime'],
            len(discussion_collection.document(disc.id).collection(
                'discussion-participants').get())
        ])
    df = pd.DataFrame(records, columns=['id', 'date', 'pcount'])
    df['date'] = df['date'].astype('datetime64[ns]')  # Create date column
    return df


ass = get_allsides_stats()


In [ ]:
# Filter by date
ass = ass[ass['date'] >= filterdate][ass['date']
                                     < filterdate + dt.timedelta(days=1)]
# print(ass[ass['pcount']>2].groupby(ass['date'].dt.date).size())  # Discussions w/ >1



# Test Chat

In [ ]:
def chat(message):
    return {'message': message,
            'creatorId': random.choice(['XZIDja4RCZVcdN8ht7As0SnrtgH2', 'NEOyBIn4CIXbIz1F0O5Zuy1wqpY2']),
            'createdDate': firestore.SERVER_TIMESTAMP,
            'messageStatus': 'active',
            #  'id': 'h9sIBJ1C0MLTlaRbDNqW',
            'collectionPath': None}


def get_chat_collection(junto_id, topic_id, discussion_id):
    return store.collection('junto').document(junto_id).collection('topics').document(
        topic_id).collection('discussions').document(discussion_id).collection('chats').document(
        'junto_chat').collection('messages')


def simulate_chats(junto_id, topic_id, discussion_id, count=100):
    chat_collection = get_chat_collection(junto_id, topic_id, discussion_id)
    letters = string.ascii_lowercase
    for i in range(count):
        msg = ''.join(random.choice(letters) for i in range(10))
        chat_collection.add(chat(msg))


def get_messages(junto_id, topic_id, discussion_id):
    chat_collection = get_chat_collection(junto_id, topic_id, discussion_id)
    return [m.to_dict() for m in chat_collection.get()]


In [ ]:
# URL: https://juntochat-dev.web.app/home/#/junto/philosophize-this/discuss/0D5ClrXaUs4x7EquSELU/T5Wdu2vGLtyNOV6SCMHM

def test_chat():
    junto_id = 'philosophize-this'
    topic_id = '0D5ClrXaUs4x7EquSELU'
    discussion_id = 'T5Wdu2vGLtyNOV6SCMHM'
    simulate_chats(junto_id, topic_id, discussion_id, 200)
    len(get_messages(junto_id, topic_id, discussion_id))


In [ ]:
document_path = 'junto/CTqYh6sW4Hueyxy2SK3d/topics/cjMTqHda5ICPrKGFewV4/discussions/SLoLPFuEIDU6YM3CDKon/discussion-participants'


def participant():
    letters = string.ascii_lowercase
    identifier = ''.join(random.choice(letters) for i in range(20))
    return {'id': identifier,
            'isPresent': True,
            'status': 'active'}


def simulate_participants(count=100):
    collection = store.collection(document_path)
    letters = string.ascii_lowercase
    for i in range(count):
        collection.add(participant())


In [ ]:
simulate_participants(count=30)


# Update LRC Conversations

In [ ]:
OLD_CONVERSATION_AGREEMENTS = '''
_These will set the tone of our conversation; participants may volunteer to take turns reading them aloud._
-   Be curious and listen to understand.
-   Show respect and suspend judgment.
-   Note any common ground as well as any differences.
-   Be authentic and welcome that from others.
-   Be purposeful and to the point.
-   Own and guide the conversation.
'''

NEW_CONVERSATION_AGREEMENTS = '''
_These will set the tone of our conversation; participants may volunteer to take turns reading them aloud._

-   **Be curious and listen to understand.**  Conversation is as much about listening as it is about talking. You might enjoy exploring how others’ experiences have shaped their values and perspectives.

-   **Show respect and suspend judgment.**  People tend to judge one another. Setting judgement aside opens you up to learning from others and makes them feel respected and appreciated. Try to truly listen, without interruption or crosstalk.

-   **Note any common ground as well as any differences.**  Look for areas of agreement or shared values that may arise and take an interest in the differing beliefs and opinions of others.

-   **Be authentic and welcome that from others.**  Share what’s important to you. Speak from your experience. Be considerate of others who are doing the same.

-   **Be purposeful and to the point.**  Do your best to keep your comments concise and relevant to the question you are answering. Be conscious of sharing airtime with other participants.

-   **Own and guide the conversation.**  Take responsibility for the quality of your participation and the conversation as a whole. Be proactive in getting yourself and others back on track if needed. Use an agreed upon signal like the “time out” sign if you feel the agreements are not being honored.
'''


In [ ]:

collection_id = '53FXvTKVnJlUPInVgDzd'
collection = store.collection('junto').document(
    collection_id).collection('topics')
topics = get_docs(collection)

# Check all have the same old conversation agreements
for doc in topics:
    docdict = doc.to_dict()
    if 'status' in docdict and docdict['status'] == 'removed':
        continue
    if 'agendaItems' not in docdict:
        continue
    agreements = docdict['agendaItems'][1]['content']
    if agreements != OLD_CONVERSATION_AGREEMENTS:
        print(docdict['title'])
        print(docdict['agendaItems'][1]['content'])
        print('\n')


def batch_data(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]


def batch_upload_junto_topics(store, topics):  # Modified
    for batched_data in batch_data(topics, 499):
        batch = store.batch()
        for doc in batched_data:
            topic_collection = store.collection('junto').document(
                collection_id).collection("topics")
            doc_ref = topic_collection.document(str(doc['docid']))
            doc.pop('docid')
            batch.set(doc_ref, doc)
        batch.commit()


In [ ]:
new_topics = []
for doc in topics:
    doc_id = doc.id
    new_topic = doc.to_dict()
    if 'status' in new_topic and new_topic['status'] == 'removed':
        continue
    if 'agendaItems' not in new_topic:
        continue
    new_topic['docid'] = doc_id
    new_topic['agendaItems'][1]['content'] = NEW_CONVERSATION_AGREEMENTS
    new_topics.append(new_topic)


In [ ]:
batch_upload_junto_topics(store, new_topics)


# Graveyard

### Create Memberships

In [ ]:
def create_memberships():
    # Used to create memberships during transition to Memberships
    junto_collection = store.collection('junto')
    member_collection = store.collection('memberships')

    def get_member_dict(userId, juntoId, status='member', firstJoined=datetime.datetime.now()):
        return {
            'userId': userId,
            'juntoId': juntoId,
            'status': status,
            'firstJoined': str(firstJoined)
        }

    for juntoref in junto_collection.stream():
        batch = store.batch()
        junto = juntoref.to_dict()
        print("%s -- %d members" %
              (junto['id'], len(junto.get('members', []))))
        juntoCreator = junto['creatorId']
        for userid in (junto.get('members', [])):
            doc_ref = member_collection.document(userid).collection(
                'junto-membership').document(junto['id'])
            batch.set(doc_ref, get_member_dict(
                userid, junto['id'], 'owner' if juntoCreator == userid else 'member'))
        batch.commit()


### DailyCo Stats

In [ ]:
# DailyCo Stats
start_ts = datetime(2021, 1, 23).replace(tzinfo=timezone.utc).timestamp()
end_ts = datetime(2021, 1, 24).replace(tzinfo=timezone.utc).timestamp()

authkey = "9aa641bbd7385aef496c3ddb545897dc6ca9d1be74fb03e1ec5c74944699b0d9"
url = "https://api.daily.co/v1/meetings"
headers = {"Authorization": "Bearer " + authkey}
querystring = {"timeframe_start": str(
    int(start_ts)), "timeframe_end": str(int(end_ts)), "limit": 100}

rooms = []
total_count = 1000
while total_count > 100:
    if len(rooms) > 0:
        querystring["starting_after"] = rooms[-1]['id']
    response = requests.request(
        "GET", url, headers=headers, params=querystring)
    total_count = json.loads(response.text)['total_count']
    rooms.extend(json.loads(response.text)['data'])


class RoomStat:
    def __init__(self, room, pcount, duration):
        self.room = room
        self.pcount = pcount
        self.duration = duration

    def to_dict(self):
        return {
            'room': self.room,
            'pcount': self.pcount,
            'duration': self.duration,
        }


roomsstats = {}
for r in rooms:
    if len(r['room']) != 25:
        continue
    rs = RoomStat(r['room'], len(r['participants']), r['duration'])
    if rs.room in roomsstats:
        if roomsstats[rs.room].pcount > rs.pcount:
            continue
    roomsstats[rs.room] = rs


df = pd.DataFrame([r.to_dict() for r in roomsstats.values()])
df['duration'] = df['duration'].div(60)
df.groupby('pcount').describe().round()
